<a href="https://colab.research.google.com/github/dsineirobarreiro/ModelER/blob/main/api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 23.5 MB/s eta 0:00:00


In [ ]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on -DCMAKE_CUDA_ARCHITECTURES=61" FORCE_CMAKE=1 pip install llama-cpp-python==0.2.62 --force-reinstall --upgrade --no-cache-dir --verbose --no-build-isolation

In [ ]:
!pip install fastapi pydantic-settings uvicorn langchain-experimental jsonref python-multipart deflate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 MB 10.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.9/206.9 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!huggingface-cli download TheBloke/Llama-2-7B-Chat-GGUF llama-2-7b-chat.Q5_K_M.gguf --local-dir ./json/ --local-dir-use-symlinks False

/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/download.py:132: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
llama-2-7b-chat.Q5_K_M.gguf: 100% 4.78G/4.78G [04:20<00:00, 18.4MB/s]
Download complete. Moving file to json/llama-2-7b-chat.Q5_K_M.gguf
json/llama-2-7b-chat.Q5_K_M.gguf


In [ ]:
from langchain_community.llms.llamacpp import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from typing import List, Optional
from langchain.output_parsers import PydanticOutputParser
from langchain.chains.llm import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_experimental.chat_models import Llama2Chat
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.messages import SystemMessage
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from jsonref import replace_refs
import json
from llama_cpp import LlamaGrammar
from typing import Literal

model_path = '/content/json/llama-2-7b-chat.Q5_K_M.gguf'

# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

class Attribute(BaseModel):
    name: str = Field(description='attribute name')
    type: str = Field(description='type of the atrtibute')

# Define your desired data structure.
class Entity(BaseModel):
    name: str = Field(description='entity name from the case scenario')
    attributes: Optional[List[Attribute]] = Field(description="colection of attributes from the entity")

class Relation(BaseModel):
    name: str = Field(description='name of the relation')
    source: str = Field(description='source entity of the relation')
    cardinality_of_source: Literal['Zero or One', 'Exactly One', 'Zero or Many', 'One or Many']
    target: str = Field(description='target entity of the relation')
    cardinality_of_target: Literal['Zero or One', 'Exactly One', 'Zero or Many', 'One or Many']


class Main(BaseModel):
    entities: List[Entity] = Field(description='Entity from the case scenario')
    relations: List[Relation] = Field(description='colection of relations from the entities')

def json_schema_with_inlining(model):
    replaced = replace_refs(model.schema(), proxies=False)
    if "$defs" in replaced:
        del replaced["$defs"]
    return json.dumps(replaced)

grammar = LlamaGrammar.from_json_schema(json_schema_with_inlining(Main))

DEFAULT_SYSTEM_PROMPT = """\
You are an expert in data modeling for creating databases. Imagine a client is talking to you in order to create a database for their business.
They will present you a scenario where your goal will be to extract entities, their attributes and their relations so the first step for creating
a database can be achieve. As it is client who asks for this modeling, you must be very accurate and effective in the extraction so work step by step.
Source and target of the relations must be names of entities and every entity must be a participant of some relationship.
"""

llm = LlamaCpp(
    model_path=model_path,
    n_ctx=4096,
    n_gpu_layers=-1,
    max_tokens=-1,
    callback_manager=callback_manager,
    verbose=True,
    streaming=True,
    grammar=grammar,
    temperature=0.2,
)

model = Llama2Chat(llm=llm)

template_messages = [
    SystemMessage(content=DEFAULT_SYSTEM_PROMPT),
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessagePromptTemplate.from_template("{input}"),
]
prompt_template = ChatPromptTemplate.from_messages(template_messages)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
chain = LLMChain(llm=model, prompt=prompt_template, memory=memory)

chains = {}
chains['llama2'] = {}
chains['generate'] = chain

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /content/json/llama-2-7b-chat.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32  

char ::= [^"\] | [\] char_1 
char_1 ::= ["\/bfnrt] | [u] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] [0-9a-fA-F] 
entities ::= [[] space entities_8 []] space 
space ::= space_38 
entities_4 ::= entities-item entities_7 
entities-item ::= [{] space entities-item-name-kv [,] space entities-item-attributes-kv [}] space 
entities_6 ::= [,] space entities-item 
entities_7 ::= entities_6 entities_7 | 
entities_8 ::= entities_4 | 
entities-item-name-kv ::= ["] [n] [a] [m] [e] ["] space [:] space string 
entities-item-attributes-kv ::= ["] [a] [t] [t] [r] [i] [b] [u] [t] [e] [s] ["] space [:] space entities-item-attributes 
entities-item-attributes ::= entities-item-attributes-0 | null 
entities-item-attributes-0 ::= [[] space entities-item-attributes-0_18 []] space 
null ::= [n] [u] [l] [l] space 
entities-item-attributes-0_14 ::= entities-item-attributes-0-item entities-item-attributes-0_17 
entities-item-attributes-0-item ::= [{] space entities-item-attributes-0-item-name-kv [,] space entities-item

llama_model_loader: - kv  13:                      tokenizer.ggml.scores arr[f32,32000]   = [0.000000, 0.000000, 0.000000, 0.0000...
llama_model_loader: - kv  14:                  tokenizer.ggml.token_type arr[i32,32000]   = [2, 3, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...
llama_model_loader: - kv  15:                tokenizer.ggml.bos_token_id u32              = 1
llama_model_loader: - kv  16:                tokenizer.ggml.eos_token_id u32              = 2
llama_model_loader: - kv  17:            tokenizer.ggml.unknown_token_id u32              = 0
llama_model_loader: - kv  18:               general.quantization_version u32              = 2
llama_model_loader: - type  f32:   65 tensors
llama_model_loader: - type q5_K:  193 tensors
llama_model_loader: - type q6_K:   33 tensors
llm_load_vocab: special tokens cache size = 3
llm_load_vocab: token to piece cache size = 0.1684 MB
llm_load_print_meta: format           = GGUF V2
llm_load_print_meta: arch             = llama
llm_load_print_meta: vocab

In [ ]:
from fastapi import FastAPI, Form
from fastapi.responses import StreamingResponse
from fastapi.middleware.cors import CORSMiddleware


app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

async def gen(model, action, prompt: str):
    async for event in chain.astream_events(
        {"input": prompt},
        version='v1'
    ):
        if event['event'] == 'on_chat_model_stream':
            yield event['data']['chunk']

@app.post('/{model}/{action}/')
async def generate(action, prompt: str = Form()):
    return StreamingResponse(gen(model, action, prompt), media_type='text/event-stream')

In [ ]:
!ngrok config add-authtoken 2esCQXildH7Ma8BWGMduStFifqX_4WvZN3id9meqREwp2ApBe

In [ ]:
import asyncio
import ngrok
import uvicorn
import nest_asyncio

import os
from google.colab import userdata


async def setup():
        ngrok.set_auth_token(userdata.get("NGROK"))
        ngrok.forward(
            8000,
            authtoken_from_env=True,
            domain="bright-akita-pleasantly.ngrok-free.app"
        )

nest_asyncio.apply()
asyncio.run(setup())
uvicorn.run(app=app)

INFO:     Started server process [6967]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     85.87.86.192:0 - "POST /llama2/generate/ HTTP/1.1" 200 OK


/usr/local/lib/python3.10/dist-packages/llama_cpp/llama.py:1138: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [6967]


import deflate

b64charset = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/"
base64_alphabet = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz-_"
source_relations = {"Zero or One": "|o", "Exactly One": "||", "Zero or Many": "}o", "One or Many": "}|"}
target_relations = {"Zero or One": "o|", "Exactly One": "||", "Zero or Many": "o{", "One or Many": "|{"}

def encode_plant_uml(diagram):
    data = diagram.encode()
    level = 6
    compressed = deflate.deflate_compress(data, level)

    chunks_8bit = ''.join([format(bits,'08b') for bits in compressed])

    chunks_6bit = [chunks_8bit[bits:bits+6] for bits in range(0,len(chunks_8bit),6)]
    padding_amount = ( 6 - len(chunks_6bit[len(chunks_6bit)-1]) )
    chunks_6bit[len(chunks_6bit)-1] += padding_amount * '0'


    encoded = ''.join([base64_alphabet[int(bits,2)] for bits in chunks_6bit])
    encoded += int(padding_amount/2) * '='

    return encoded

def create_uml_diagram(data):
    uml = '@startuml\n'
    for ent in data['entities']:
        uml += 'entity ' + ent['name'] + '{\n'
        uml += '* ' + ent['name'] + 'ID: number <<generated>>\n--\n'
        for attr in ent['attributes']:
            if('id' in attr['name'].lower()): continue
            uml += '* ' + attr['name'] + ': ' + attr['type'] + '\n'
        uml += '}\n\n'

    for rel in data['relations']:
        uml += rel['source'] + ' ' + source_relations[rel['cardinality_of_source']] + '--' + target_relations[rel['cardinality_of_target']] + rel['target'] + '\n'

    uml += '@enduml'
    return encode_plant_uml(uml)

def create_mermaid_diagram(data):
    merm = 'erDiagram\n'
    for ent in data['entities']:
        merm += ent['name'] + '{\n'
        merm += 'int ' + ent['name'] + 'ID PK\n'
        for attr in ent['attributes']:
            attr['name'].replace(' ', '_')
            if('id' in attr['name'].lower()): continue
            merm += attr['type'] + ' ' + attr['name'] + '\n'
        merm += '}\n\n'

    for rel in data['relations']:
        merm += rel['source'] + ' ' + source_relations[rel['cardinality_of_source']] + '--' + target_relations[rel['cardinality_of_target']] + rel['target'] + ': ' + rel['name'] + '\n'

    return merm

```
# Tiene formato de código

```
# Tiene formato de código
```


```

